# Example
This notebook illustrates how the sleepwellbaby package can be used to get sleep state predictions using mock data.

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from tqdm import tqdm

from sleepwellbaby.data import (
    compute_reference_values,
    convert_to_payload,
    generate_mock_signalbase_data,
)
from sleepwellbaby.model import get_prediction, load_model


## Prepare data
Generate some mock data (same format as SignalBase with 1Hz)

In [ ]:
df = generate_mock_signalbase_data().sort_values(by='datetime').set_index('datetime')

Compute reference values using rolling windows

In [ ]:
df = compute_reference_values(df, freq=1)

## Get predictions 
We make a prediction every one minute in the below example

In [ ]:
# Get indices of times at which you want to get an SWB value (we only compute it at whole minutes)
t_range_swb = pd.date_range(start=df.index.min().round(freq='min'), end=df.index.max().round(freq='min'), freq='1min')

In [ ]:
# Helper function to get predictions
def get_SWB(df, indices):
    model, model_support_dict = load_model()
    ref_columns = [c for c in df.columns if ('mean' in c) or ('std' in c)]
    for ix, t in enumerate(tqdm(indices, desc="Calculating SWB")):
        # Get the right timestamps for SWB, one every 2.5 seconds
        # For 1Hz data we round so we do not have to interpolate
        timestamps = pd.date_range(t - pd.Timedelta(8, 'm'), t, freq='2500ms', inclusive='right').round('1s')
        temp = df.reindex(index=timestamps)  # sets rows with missing indices to nan
        row = temp.iloc[-1]
        assert row.name == t

        # Check if reference values are defined
        if row[ref_columns].isna().any():
            pred = 'ineligible'
            proba_dict = {'AS': -1, 'QS': -1, 'W': -1}
        else:
            payload = convert_to_payload(temp, birth_date='2000-01-01')
            pred, proba_dict = get_prediction(payload, model, model_support_dict)
        columns =list(proba_dict.keys())
        if ix == 0:
            df.loc[:, 'prediction'] = None
            df.loc[:, columns] = None
        df.loc[t, 'prediction'] = pred
        df.loc[t, columns] = pd.DataFrame(proba_dict, index=[0,]).iloc[0]
    return df, columns

Return predictions (takes a couple of minutes for a single day with one prediction every minute)

In [ ]:
df, columns = get_SWB(df, t_range_swb)

Drop rows without a prediction

In [ ]:
df.dropna(subset='prediction', inplace=True)

In [ ]:
df.tail(10)

#